In [2]:
d = pd.read_pickle('A123_OCV_vs_SOC_data.pkl')

In [4]:
d.keys()

dict_keys(['P25', 'N05', 'N15', 'N25', 'P05', 'P15', 'P35', 'P45'])

__Nested Key Finding Function__

In [6]:
def nest_key(d, indent=0):
    
    for key, value in d.items():
        
        print('\t' * indent + f'{key}')
        
        if isinstance(value, dict):
            
            nest_key(value, indent+1)